In [5]:
import cv2
import time, os
import threading
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt

from situtils import FPSTimes

In [3]:
class VideoWriter(FPSTimes):
    # precise timing https://stackoverflow.com/questions/42565297/precise-loop-timing-in-python
    
    default_cfg = {
        'fps': 20,
        'file_path': 'test_video.avi',
    }
    
    def __init__(self, status, video_stream, cfg):
        super(VideoWriter, self).__init__()
        
        self.cfg = cfg
        self.video_stream = video_stream  # cv2 video stream
        self.stopped = False
        self.status = status
        
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        res_x, res_y = int(video_stream.stream.get(3)), int(video_stream.stream.get(4))
        self.out = cv2.VideoWriter(cfg['file_path'], fourcc, cfg['fps'], (res_x, res_y))

    @property
    def latency(self):
        return 1.0/self.cfg['fps']
        
    def start(self):
        self._th = threading.Thread(target=self.update, args=())
        self._th.start()
    
    def stop(self):
        self.stopped = True
        time.sleep(0.2)   # wait until device is released
        self._th.join()
        print('Video writer stopped')
        
    def update(self):
        next_frame = time.time() + self.latency
        
        while not self.stopped:
            if self.status.value == 2:
                if time.time() < next_frame:
                    continue

                #frame = self.video_stream.read()
                frame = self.video_stream.frame_with_infos
                if frame is not None:
                    self.count()  # count FPS
                    self.out.write(frame)

                    frames_missed = np.floor((time.time() - next_frame) * self.cfg['fps'])
                    next_frame += self.latency * frames_missed + self.latency
            else:
                time.sleep(0.05)
            
        self.out.release()

### Testing video recording

In [13]:
#import nbimporter
#from camera import WebcamStream

# let's use a webcam stream
vs = WebcamStream(WebcamStream.default_cfg)
vs.start()  # stream runs in a separate thread

# recording status: 1 - idle, 2 - recording
status = mp.Value('i', 1)

# start/stop recording by pressing 's'
vw = VideoWriter(status, vs, VideoWriter.default_cfg)
vw.start()

try:
    while True:
        frame = vs.read()
        if frame is not None:
            color = (127, 255, 0) if status.value == 1 else (0, 0, 255)
            frame = cv2.circle(frame, (20, 20), 15, color, -1)
            cv2.imshow('Webcam', frame)

        k = cv2.waitKey(33)
        if k == ord('q'):
            break
            
        if k == ord('s'):
            status.value = 2 if status.value == 1 else 1
finally:
    cv2.destroyAllWindows()
    vs.stop(), vw.stop()

Webcam stream 960.0:720.0 at 20.00 FPS started
Camera released
Video writer stopped
